In [1]:
# ایمپورت کردن کتابخانه ها
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import sklearn 

In [2]:
# خواندن دیتا
data = pd.read_csv("training.1600000.processed.noemoticon.csv" , encoding='ISO-8859-1')

In [5]:
data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [6]:
# آشنایی بیشتر با دیتا
data['0'].value_counts()

4    800000
0    799999
Name: 0, dtype: int64

In [7]:
data['NO_QUERY'].value_counts()

NO_QUERY    1599999
Name: NO_QUERY, dtype: int64

In [8]:
data['_TheSpecialOne_'].value_counts()[:10]

lost_dog           549
webwoke            345
tweetpet           310
SallytheShizzle    281
VioletsCRUK        279
mcraddictal        276
tsarnick           248
what_bugs_u        246
Karen230683        238
DarkPiano          236
Name: _TheSpecialOne_, dtype: int64

In [9]:
# تغییر نام ستون ها
data = data.rename(columns={"0":"label" , "Mon Apr 06 22:19:45 PDT 2009":"created_at","NO_QUERY":"status",
                     "_TheSpecialOne_":"user_name" ,
                     "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D":"text", 
                    "1467810369":"id"})

In [10]:
data = data.rename(columns = {"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D":"text"})

In [11]:
data.head()

,label,id,created_at,status,user_name,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [12]:
print(len(data))

1599999


In [13]:
# حذف داده های تکراری
data = data.drop_duplicates()
print(len(data))

1599999


In [14]:
# حذف ستون های کم اهمیت تر
data = data.drop(columns= ["status"])
data.head()

,label,id,created_at,user_name,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,joy_wolf,@Kwesidei not the whole crew


In [13]:
import nltk

In [15]:
# ایجاد دیتافریم شامل هر دو نوع توییت
d0 = data[data['label'] == 0 ][:200000]
d4 = data[data['label'] == 4][:200000]
dd = pd.concat([d0,d4]).reset_index()
X = dd['text'].values
Y = dd['label'].values

In [16]:
# bag of words 
# ایمپورت کردن ماژول های مورد نیاز
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
vectorizer_bag = CountVectorizer(tokenizer=word_tokenize , stop_words=['the'])

In [104]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rayan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [17]:
# ایجاد داده های آموزش ، اعتبار و تست
from sklearn.model_selection import train_test_split
x_train_ , x_test , y_train_ , y_test = train_test_split(X , Y , test_size=0.3)

In [18]:
x_train , x_val , y_train , y_val = train_test_split(x_train_ , y_train_ , test_size=0.3)

# Bag Of Words

In [19]:
# آموزش مدل نیو بیز با ورودی کیسه کلمات
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
model_naive_bag = Pipeline([
    ('vectorizer' , CountVectorizer(min_df=0.0 , lowercase=False)),
    ('nb' , MultinomialNB())
])
model_naive_bag.fit(x_train , y_train)

Pipeline(steps=[('vectorizer', CountVectorizer(lowercase=False, min_df=0.0)),
                ('nb', MultinomialNB())])

In [20]:
print("accuracy naive BOW : " , model_naive_bag.score(x_val , y_val))

accuracy naive BOW :  0.7694166666666666


In [21]:
# آموزش مدل آدابوست با ورودی کیسه کلمات
from sklearn.ensemble import AdaBoostClassifier
model_ada_bag = Pipeline([
    ('vect_ada_bag' , CountVectorizer(min_df=0.0 , lowercase=False)),
    ('ada_bag' , AdaBoostClassifier())
])
model_ada_bag.fit(x_train , y_train)

Pipeline(steps=[('vect_ada_bag', CountVectorizer(lowercase=False, min_df=0.0)),
                ('ada_bag', AdaBoostClassifier())])

In [22]:
print("accuracy AdaBoost BOW : " , model_ada_bag.score(x_val , y_val))

accuracy AdaBoost BOW :  0.6711785714285714


In [23]:
# آموزش مدل رگرسیون لاجیستیک با ورودی کیسه کلمات
from sklearn.linear_model import LogisticRegression
model_logreg_bag = Pipeline([
    ('vect_logreg_bag' , CountVectorizer(min_df=0.0 , lowercase=False)),
    ('logreg_bag' , LogisticRegression())
])
model_logreg_bag.fit(x_train,y_train)

C:\Users\Rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vect_logreg_bag',
                 CountVectorizer(lowercase=False, min_df=0.0)),
                ('logreg_bag', LogisticRegression())])

In [24]:
print("accuracy logreg BOW : " , model_logreg_bag.score(x_val , y_val))

accuracy logreg BOW :  0.7830119047619047


In [25]:
# TFIDF
# ایمپورت کردن ماژول های مورد نیاز
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=word_tokenize)

In [26]:
# آموزش مدل نیو بیز با ورودی تی اف آی دی اف
model_naive_tf = Pipeline([
    ('vec_tf' , TfidfVectorizer(tokenizer=word_tokenize , min_df=0.0 , lowercase=False)) , 
    ('nb_tf' , MultinomialNB())
])
model_naive_tf.fit(x_train , y_train)

C:\Users\Rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vec_tf',
                 TfidfVectorizer(lowercase=False, min_df=0.0,
                                 tokenizer=<function word_tokenize at 0x0000015D8790E710>)),
                ('nb_tf', MultinomialNB())])

In [27]:
print("accuracy naive tfidf : " , model_naive_tf.score(x_val , y_val))

accuracy naive tfidf :  0.7666190476190476


In [163]:
# آموزش مدل آدابوست با ورودی تی اف آی دی اف
model_ada_tf = Pipeline([
    ('vect_ada_tf' , TfidfVectorizer(tokenizer=word_tokenize , min_df=0.0 , lowercase=False)),
    ('ada_tf' , AdaBoostClassifier())
])

model_ada_tf.fit(x_train, y_train)

C:\Users\Rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect_ada_tf',
                 TfidfVectorizer(lowercase=False, min_df=0.0,
                                 tokenizer=<function word_tokenize at 0x000001B37D3C8EE0>)),
                ('ada_tf', AdaBoostClassifier())])

In [164]:
print("accuracy AdaBoost tfidf : " , model_ada_tf.score(x_val , y_val))

accuracy AdaBoost tfidf :  0.6978452380952381


In [28]:
# آموزش مدل رگرسیون لاجیستیک با ورودی تی اف آی دی اف
model_logreg_tf = Pipeline([
    ('vect_logreg_tf' , TfidfVectorizer(tokenizer=word_tokenize , min_df=0.0 , lowercase=False)),
    ('logreg_tf'  , LogisticRegression())
])
model_logreg_tf.fit(x_train , y_train)

C:\Users\Rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\Rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vect_logreg_tf',
                 TfidfVectorizer(lowercase=False, min_df=0.0,
                                 tokenizer=<function word_tokenize at 0x0000015D8790E710>)),
                ('logreg_tf', LogisticRegression())])

In [29]:
print("accuracy logreg tfidf : " , model_logreg_tf.score(x_val,y_val))

accuracy logreg tfidf :  0.7943095238095238


In [30]:
# با توجه به نتایج به دست آمده ، آموزش مدل
# logistic regression 
# با روش
# tfidf
# بهترین نتیجه را میدهد
# پس برای دیتای تست ، از همین روند استفاده میکنیم

In [31]:
# دقت مدل نهایی بر روی داده تست
print("accuracy logreg tfidf on test data : " , model_logreg_tf.score(x_test,y_test))

accuracy logreg tfidf on test data :  0.78925


In [172]:

from transformers import BertModel , BertConfig
bert_configuration = BertConfig()
bert_model = BertModel(bert_configuration)
bert_configuration = bert_model.config

In [173]:
from transformers import AutoTokenizer , BertModel , TFBertTokenizer
tokenizer_pre = AutoTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

C:\Users\Rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Rayan\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# آموزش مدل با استفاده از برت
from transformers import TFBertModel , TFBertTokenizer
tf_bert_token = TFBertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
inp_bert = tf_(list(x_train) , return_tensors='tf')
outputs = bert_model(inp_bert)

In [193]:
from tensorflow import keras_nlp

features = x_train
labels = y_train

# Pretrained classifier.
classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_base_en_uncased",
    num_classes=2,
)
classifier.fit(x=features, y=labels, batch_size=2)
bert_pred_val = classifier.predict(x=x_val, batch_size=2)

ImportError: cannot import name 'keras_nlp' from 'tensorflow' (C:\Users\Rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\__init__.py)

In [ ]:
# حذف قسمت های غیر مورد نیاز برای کاهش استفاده از حافظه

In [39]:
del model_ada_bag

In [40]:
del model_naive_tf

In [41]:
del model_naive_bag

# Part3

# two best classifiers of the second part on the dataset of the first part

In [80]:
# خواندن داده های توییت ها
tweets = pd.read_csv("tweets.csv")

In [81]:
# انتخاب 9 هزار داده برای تست مدل ها
msk_tweet = np.random.rand(len(tweets)) < 0.001

In [82]:
test_tweet = tweets[msk_tweet]
print(len(test_tweet))

9019


In [83]:
# حذف برای کاهش حافظه
del tweets

In [84]:
# پیش بینی یافته با مدل لاجیستیک رگرسیون و ورودی کیسه کلمات
pred_tweets_bag = model_logreg_bag.predict(test_tweet['text'].values)

In [85]:
pd.DataFrame(pred_tweets_bag).value_counts()

4    5595
0    3424
dtype: int64

In [86]:
# پیش بینی یافته با مدل لاجیستیک رگرسیون و ورودی تی اف آی دی اف
pred_tweets_tf = model_logreg_tf.predict(test_tweet['text'].values)

In [87]:
pd.DataFrame(pred_tweets_tf).value_counts()

4    5022
0    3997
dtype: int64

In [88]:
# مشخص کردن اندیس های داده تست که دو مدل ، پیش بینی متفاوت دارند
msk_diff = pd.DataFrame({'val': pred_tweets_bag})['val'] == pd.DataFrame({'val': pred_tweets_tf})['val']
msk_diff

0        True
1        True
2        True
3       False
4        True
        ...  
9014    False
9015     True
9016     True
9017     True
9018     True
Name: val, Length: 9019, dtype: bool

In [97]:
# اضافه کردن پیش بینی مدل ها و حذف ستون های به درد نخور
test_tweet = test_tweet.reset_index()
test_tweet = pd.DataFrame(test_tweet['text'])
test_tweet

In [91]:
test_tweet.insert(0,"bag" , pred_tweets_bag)
test_tweet.insert(0,"tf" , pred_tweets_tf)

In [92]:
test_tweet

,tf,bag,text
0,0,0,Invesco Ltd. $IVZ Position Reduced by Ameripri...
1,4,4,Incyte Co. $INCY Trading Up 6.9% on Insider Bu...
2,0,0,Somewhat Negative Media Coverage Somewhat Like...
3,0,4,RT @SmCapMarketPl: $ABVG Acquisition NEWS!$APT...
4,4,4,RT @wildsidekittens: Survey:Do you know how to...
...,...,...,...
9014,0,4,Critical Comparison: KalVista Pharmaceuticals ...
9015,0,0,Comparing Veeva Systems $VEEV &amp; Allscripts...
9016,4,4,"RT @SAI: Here are big, beautiful photos of the..."
9017,4,4,3-star analyst Mark Delaney from Goldman Sachs...


In [93]:
# توییت هایی که پیش بینی دو مدل برای آنها متفاوت هست
diff_tweets = test_tweet[~msk_diff]
diff_tweets

,tf,bag,text
3,0,4,RT @SmCapMarketPl: $ABVG Acquisition NEWS!$APT...
5,4,0,CyrusOne CEO banks company's growth on new $60...
9,4,0,Moog Inc $MOG.A Raised to Buy at Zacks Investm...
12,4,0,RT @lithiumforum: Great report! #Energy transi...
14,0,4,RT @BAMMERPICS: #press #otcnews ???? NNSR$ECOB...
...,...,...,...
8991,0,4,Select Comfort Corp. $SCSS Reaches $29.68 Afte...
8999,4,0,RT @howardlindzon: Apple cellular watch....rid...
9000,0,4,NiSource Inc. $NI Reaches $26.71 After 9.00% D...
9004,0,4,$AAPL shares now down 0.7 percent during #Appl...


In [95]:
# نمایش متن توییت هایی که پیش بینی دو مدل برایشان متفاوت هست
for i in diff_tweets.index[:10] :
    print(diff_tweets.loc[i]['text'])
    print(diff_tweets.loc[i]['bag'] , diff_tweets.loc[i]['tf'])
    print("--------------------------------------------------------")

RT @SmCapMarketPl: $ABVG Acquisition NEWS!$APTY $EFIN $DCTH $EXAD $ENRT $SDVI $RITE $GRPV $IHSI @ZeroTripple#OTCmkts #Pennystocks $GIGL $…
4 0
--------------------------------------------------------
CyrusOne CEO banks company's growth on new $600M data center campus in Allen: https://t.co/O817pZkHi1 (subscriber content) $CONE
0 4
--------------------------------------------------------
Moog Inc $MOG.A Raised to Buy at Zacks Investment Research https://t.co/nsqqRInfOX
0 4
--------------------------------------------------------
RT @lithiumforum: Great report! #Energy transition supported by @German Energiewende. #Battery production boom as turbocharger! $TSLAhttps…
0 4
--------------------------------------------------------
RT @BAMMERPICS: #press #otcnews ???? NNSR$ECOB $CBIS $SING $NWAV $ICNB $MMHC $PGUS $MYDX $SANP $CHRO $OWCP $KOSK $CDII $ABCE $ENRT… 
4 0
--------------------------------------------------------
Trade with the best indicators https://t.co/lEGQ2dcYMD ???? $BSX $DAL $

In [96]:
# همان طور که مشاهده میکنیم ، به نظر میرسد که مدلی که از 
# tfidf 
# استفاده میکند ، عملکرد بهتری روی این دیتاها دارد . برای اینکه در کل ، عملکرد مدل ها بهتر شود چند کار میتوان 
# انجام داد :
# 1. استفاده از استاپ ورد های زیاد برای تجزیه و تحلیل بهتر داده ها 
# 2. افزایش داده های آموزشی ( که به دلیل محدودیت امکانات این کار برای من ممکن نبود )
# 3. از چند مدل استفاده کنیم و خروجی های آنها را دریافت کرده و از رای اکثریت استفاده کنیم 
# 4. در مدل های از پیش آموزش یافته ، اگر قسمتی از ضرایب را فریز کنیم و قسمت دیگر را با توجه به دیتاهای آموزش
# فاین تیون کنیم ، میتواند عملکرد خیلی خوبی پیدا کند .
# 5. در مدل های عمیق ، استفاده از لایه های پرونینگ و ... که پارامتر ها را کاهش میدهند ، استفاده کنیم

In [ ]:
# 